In [46]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import yfinance as yf
import time
from pathlib import Path
import os
import openpyxl
from sec_cik_mapper import StockMapper
from datetime import datetime, timedelta


In [2]:
# Get the current working directory
current_dir = os.getcwd()

# Resolve the parent directories
project_general_path = Path(current_dir).resolve()
print(project_general_path)

C:\Users\Adam Krupa\OneDrive\Pulpit\Investing\earnings-report-analysis


In [ ]:
def get_tickers():
    # URL of the website containing the S&P 500 tickers
    url = 'https://www.slickcharts.com/sp500'

    # Fetch the page content with headers to avoid being blocked
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table', {'class': 'table table-hover table-borderless table-sm'})
        tickers = []
        
        # Extract tickers from the table
        if table:
            for row in table.find('tbody').find_all('tr'):
                columns = row.find_all('td')
                ticker = columns[2].text.strip()  # The third column contains the ticker symbol
                tickers.append(ticker)
        else:
            print("Table not found in the page")
    else:
        print("Failed to fetch S&P 500 tickers")
        tickers = []

    return tickers
    


In [ ]:
tickers = get_tickers()

In [37]:
tickers_short = tickers[:10]

In [39]:
def get_dates(tickers):
    # Initialize an empty DataFrame to store valid earnings data
    data = pd.DataFrame(columns=['Ticker', 'Date'])

    # Loop through each stock ticker
    for ticker_symbol in tickers:
        ticker = yf.Ticker(ticker_symbol)
        # Fetch the earnings history (past earnings reports)
        earnings_history = ticker.earnings_dates
        
        # Skip if no earnings data is available
        if earnings_history is None:
            print(f"{ticker_symbol}: possibly delisted; no earnings dates found")
            continue

        # Iterate through each earnings report and determine the correct date classification
        for index, row in earnings_history.iterrows():
            report_time = pd.to_datetime(row.name)  # Access the index (which is the earnings date) and convert it to datetime
            # if report_time.hour >= 16:  # After 4 PM, classify as next day because the trade based on this knowledge can effectively only be executed the day after
            #     adjusted_date = (report_time + pd.Timedelta(days=1)).date()
            # else:  # Otherwise, use the same day
            #     adjusted_date = report_time.date()

            # Add each adjusted earnings date as a separate row in the DataFrame
            new_row = {'Ticker': ticker_symbol, 'Date': report_time}
            data = pd.concat([data, pd.DataFrame([new_row])], ignore_index=True)

    return data


In [42]:
def get_cik(data):
    mapper = StockMapper()
    ticker_to_cik = mapper.ticker_to_cik

    # Ensure tickers are in uppercase to match the mapping keys
    data['Ticker'] = data['Ticker'].str.upper()

    # Map the 'Ticker' column to CIK numbers
    data['CIK'] = data['Ticker'].map(ticker_to_cik)

    return data


In [40]:
data = get_dates(tickers_short)

C:\Users\Adam Krupa\AppData\Local\Temp\ipykernel_141800\2737740090.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat([data, pd.DataFrame([new_row])], ignore_index=True)
BRK.B: $BRK.B: possibly delisted; no earnings dates found


BRK.B: possibly delisted; no earnings dates found


In [43]:
data = get_cik(data)

In [ ]:
# Replace with your CIK and timestamp of interest
cik = '0000320193'  # Example CIK for Apple Inc.
date_with_time = '2024-10-30 20:00:00-04:00'  # Timestamp with timezone

# Parse the date with timezone information
date_time = datetime.fromisoformat(date_with_time)

# Derive the two possible dates based on the time
date_day_1 = date_time.strftime('%Y-%m-%d')
date_day_2 = (date_time + timedelta(days=1)).strftime('%Y-%m-%d')

# Ensure the CIK is 10 digits with leading zeros
cik_padded = cik.zfill(10)

# SEC EDGAR Submissions URL
url = f'https://data.sec.gov/submissions/CIK{cik_padded}.json'

# Set a custom User-Agent to avoid request blocking
headers = {'User-Agent': 'Name Surname (example@gmail.com)'}

# Fetch the submissions data
response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    filings = data.get('filings', {}).get('recent', {})
    filing_dates = filings.get('filingDate', [])
    form_types = filings.get('form', [])
    document_links = filings.get('primaryDocument', [])

    # Check for filings on both dates
    print(f"Checking filings for dates: {date_day_1} and {date_day_2}")
    for date, form, doc in zip(filing_dates, form_types, document_links):
        if date in [date_day_1, date_day_2]:
            print(f'Form Type: {form}')
            print(f'Filing Date: {date}')
            print(f'Document Link: https://www.sec.gov/Archives/edgar/data/{cik_padded}/{doc}')
            print('---')
else:
    print(f'Failed to fetch data: {response.status_code}')


Checking filings for dates: 2024-10-30 and 2024-10-31
Form Type: 8-K
Filing Date: 2024-10-31
Document Link: https://www.sec.gov/Archives/edgar/data/0000320193/aapl-20241031.htm
---


In [45]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

print(data)

    Ticker                      Date         CIK
0     AAPL 2025-10-28 20:00:00-04:00  0000320193
1     AAPL 2025-07-29 20:00:00-04:00  0000320193
2     AAPL 2025-04-29 20:00:00-04:00  0000320193
3     AAPL 2025-01-29 19:00:00-05:00  0000320193
4     AAPL 2024-10-30 20:00:00-04:00  0000320193
5     AAPL 2024-07-31 20:00:00-04:00  0000320193
6     AAPL 2024-05-01 20:00:00-04:00  0000320193
7     AAPL 2024-01-31 19:00:00-05:00  0000320193
8     AAPL 2023-11-01 20:00:00-04:00  0000320193
9     AAPL 2023-08-02 20:00:00-04:00  0000320193
10    AAPL 2023-05-03 20:00:00-04:00  0000320193
11    AAPL 2023-02-01 19:00:00-05:00  0000320193
12    NVDA 2025-11-18 19:00:00-05:00  0001045810
13    NVDA 2025-11-17 19:00:00-05:00  0001045810
14    NVDA 2025-08-25 20:00:00-04:00  0001045810
15    NVDA 2025-05-19 20:00:00-04:00  0001045810
16    NVDA 2025-02-25 19:00:00-05:00  0001045810
17    NVDA 2024-11-19 19:00:00-05:00  0001045810
18    NVDA 2024-08-27 20:00:00-04:00  0001045810
19    NVDA 2024-05-2